In [ ]:
# This cell tells python to use the local version of CVToolkit instead of the installed one.
# Use for testing purposes until branch feature/BCV-970-oor-metrics is merged.
import sys
import os

module_path = os.path.abspath(os.path.join("../../CVToolkit"))
if module_path not in sys.path:
    sys.path.insert(0, module_path)

In [ ]:
import sys
import pandas as pd

sys.path.append("../..")

from objectherkenning_openbare_ruimte.performance_evaluation_pipeline.metrics.per_pixel_stats import (
    EvaluatePixelWise,
)
from objectherkenning_openbare_ruimte.performance_evaluation_pipeline.metrics.metrics_utils import (
    ObjectClass,
    BoxSize,
)

In [ ]:
# Input: ground truth and predictions
gt_path = "../../datasets/oor/processed-first-official-training-dataset-oor/labels/val"
pred_path = "../../experiments/pred_val_yolov8m_1280/labels"

# Size of the images (width, height)
img_shape = (3840, 2160)

In [ ]:
# Compute performance over classes and bounding box sizes

evaluator = EvaluatePixelWise(gt_path, pred_path, img_shape)


# Compute the Total Blurred Area results for the person & licence plate classes

tba_results = evaluator.collect_results_per_class_and_size(
    classes=[ObjectClass.person, ObjectClass.license_plate]
)
evaluator.store_tba_results(results=tba_results)

In [ ]:
# Compute per pixel results 

per_pixel_results = evaluator.collect_results_per_class_and_size(
    classes=ObjectClass,
    single_size_only=True,
)

pd.DataFrame(data=per_pixel_results).transpose()

In [ ]:
import json
from typing import List, Tuple
from pycocotools.coco import COCO
from objectherkenning_openbare_ruimte.performance_evaluation_pipeline.metrics.custom_coco_evaluator import CustomCOCOeval


def coco_evaluation(
    coco_annotations_json: str,
    coco_predictions_json: str,
    predicted_img_shape: Tuple[int, int],
    class_ids: List[int] = [0, 1, 2, 3, 4],
    class_labels: List[str] = ["person", "license plate", "container", "mobile toilet", "scaffolding"],
    print_summary=True,
):
    """
    Runs COCO evaluation on the output of YOLO validation

    Parameters
    ----------
    coco_annotations_json: annotations in the COCO format compatible with yolov5. Comes from the metadata pipeline
    coco_predictions_json: predictions in COCO format of the yolov5 run.
    metrics_metadata: info about image sizes and areas for sanity checks.

    Returns
    -------

    """
    COCO_gt = COCO(coco_annotations_json)  # init annotations api
    try:
        COCO_dt = COCO_gt.loadRes(coco_predictions_json)  # init predictions api
    except FileNotFoundError:
        raise Exception(
            f"The specified file '{coco_predictions_json}' was not found."
            f"The file is created at the above path if you run yolo validation with"
            f"the --save-json flag enabled."
        )
    evaluation = CustomCOCOeval(COCO_gt, COCO_dt, "bbox")

    # Opening JSON file
    with open(coco_annotations_json) as f:
        data = json.load(f)

    height = data["images"][0]["height"]
    width = data["images"][0]["width"]
    if (
        height != predicted_img_shape[0]
        or width != predicted_img_shape[1]
    ):
        print(
            f"You're trying to run evaluation on images of size {height} x {width}, "
            "but the coco annotations have been generated from images of size "
            f"{predicted_img_shape[0]} x {predicted_img_shape[1]}."
            "Why is it a problem? Because the coco annotations that the metadata produces and the "
            " *_predictions.json produced by the yolo run are both in absolute format,"
            "so we must compare use the same image sizes."
            "Solutions: 1. Use images for validation that are the same size as the ones you used for the "
            "annotations. 2. Re-compute the coco_annotations_json using teh right image shape."
        )

    image_names = [image["id"] for image in data["images"]]
    evaluation.params.imgIds = image_names  # image IDs to evaluate
    evaluation.params.catIds = class_ids
    class_labels = [class_labels[i] for i in class_ids]
    evaluation.params.catLbls = class_labels

    img_area = height * width

    areaRng = []
    for areaRngLbl in evaluation.params.areaRngLbl:
        aRng = {"areaRngLbl": areaRngLbl}
        for obj_cls in ObjectClass:
            box = BoxSize.from_objectclass(obj_cls).__getattribute__(areaRngLbl)
            aRng[obj_cls.value] = (box[0]*img_area, box[1]*img_area)
        areaRng.append(aRng)

    evaluation.params.areaRng = areaRng

    evaluation.evaluate()
    evaluation.accumulate()
    evaluation.summarize(print_summary=print_summary)

    return evaluation

In [ ]:
predicted_img_shape = (3840, 2160)

coco_annotations = "../../datasets/oor/processed-first-official-training-dataset-oor/coco_annotations_val.json"
coco_predictions = "../../experiments/val_val_yolov8m_1280/predictions.json"

eval = coco_evaluation(
    coco_annotations_json=coco_annotations,
    coco_predictions_json=coco_predictions,
    predicted_img_shape=predicted_img_shape,
    class_ids=[0,1,2],
    print_summary=True,
)

In [ ]:
eval.eval["recall"].shape

In [ ]:
eval.stats